In [ ]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import os

# Load the dataset
def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
    df.dropna(inplace=True)
    return df

# Define the Strategy
class DualMA_Strategy(Strategy):
    fast_ma = 10
    slow_ma = 50
    stop_loss_pct = 0.02
    take_profit_pct = 0.04

    def init(self):
        price = self.data.Close
        self.sma_fast = self.I(SMA, price, self.fast_ma)
        self.sma_slow = self.I(SMA, price, self.slow_ma)

    def next(self):
        price = self.data.Close[-1]
        if crossover(self.sma_fast, self.sma_slow):
            self.buy(sl=price * (1 - self.stop_loss_pct), tp=price * (1 + self.take_profit_pct))
        elif crossover(self.sma_slow, self.sma_fast):
            self.position.close()

# Run backtest and extract trades
def run_backtest_and_save_trades(file_path, output_file):
    df = load_data(file_path)
    bt = Backtest(df, DualMA_Strategy, cash=100000, commission=0.001, exclusive_orders=True)
    stats = bt.run()

    trades = stats['_trades']
    if trades is not None:
        trades = trades[['EntryBar', 'ExitBar', 'EntryPrice', 'ExitPrice', 'Size', 'PnL']]
        trades['name'] = os.path.splitext(os.path.basename(file_path))[0]
        trades['entry determine'] = 'SMA crossover'
        trades['exit determine'] = 'Reverse crossover'

        trades['entry price'] = trades['EntryPrice']
        trades['exit price'] = trades['ExitPrice']
        trades['quantity'] = trades['Size']
        trades['profit/loss value'] = trades['PnL']

        trades['entry datetime'] = pd.to_datetime(df.index[trades['EntryBar']]).strftime('%d %b %y')
        trades['exit datetime'] = pd.to_datetime(df.index[trades['ExitBar']]).strftime('%d %b %y')

        # Reorganize columns
        trades = trades[['name', 'entry determine', 'entry price', 'exit determine', 'exit price',
                         'quantity', 'profit/loss value', 'entry datetime', 'exit datetime']]

        # Append to output file
        if not os.path.isfile(output_file):
            trades.to_csv(output_file, index=False)
        else:
            trades.to_csv(output_file, mode='a', header=False, index=False)

        print(f"Trades saved to {output_file} for {file_path}")
    else:
        print(f"No trades generated for {file_path}")

# Perform backtest for multiple datasets
def cross_validate_and_save_trades(file_paths, output_file):
    if os.path.exists(output_file):
        os.remove(output_file)  # Remove existing file to start fresh

    for file_path in file_paths:
        run_backtest_and_save_trades(file_path, output_file)

# List of file paths
file_paths = [
    "/content/Adani Power.csv",
    "/content/Tata Motors.csv",
    "/content/Adani Enterprises.csv",
    "/content/Adani Ports & SEZ.csv",
    "/content/Axis Bank.csv",
    "/content/DLF.csv",
    "/content/Hindalco Industries.csv",
    "/content/ICICI Bank.csv",
    "/content/Power Finance Corporation.csv",
    "/content/Punjab National Bank.csv",
    "/content/Rural Electrification Corporation.csv",
    "/content/State Bank of India.csv"
]

output_file = "trade_results.csv"
cross_validate_and_save_trades(file_paths, output_file)
print(f"\nFinal trade results saved to {output_file}")


In [ ]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import os

def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
    df.dropna(inplace=True)
    return df

class DualSMA_Strategy(Strategy):
    fast_ma = 10
    slow_ma = 50
    stop_loss_pct = 0.02
    take_profit_pct = 0.05
    position_size = 0.2
    min_volatility = 0.005

    def init(self):
        price = self.data.Close
        self.sma_fast = self.I(SMA, price, self.fast_ma)
        self.sma_slow = self.I(SMA, price, self.slow_ma)

    def next(self):
        price = self.data.Close[-1]
        price_change = abs(self.data.Close[-1] - self.data.Close[-2]) / self.data.Close[-2]

        if price_change < self.min_volatility:
            return

        if crossover(self.sma_fast, self.sma_slow):
            stop_loss = price * (1 - self.stop_loss_pct)
            take_profit = price * (1 + self.take_profit_pct)
            self.buy(size=self.position_size, sl=stop_loss, tp=take_profit)

        elif crossover(self.sma_slow, self.sma_fast):
            self.position.close()


def run_backtest_and_save_trades(file_path, output_file):
    df = load_data(file_path)
    bt = Backtest(df, DualSMA_Strategy, cash=100000, commission=0.001, exclusive_orders=True)
    stats = bt.run()

    trades = stats['_trades']
    if trades is not None:
        trades = trades[['EntryBar', 'ExitBar', 'EntryPrice', 'ExitPrice', 'Size', 'PnL']]
        trades['name'] = os.path.splitext(os.path.basename(file_path))[0]
        trades['entry datetime'] = pd.to_datetime(df.index[trades['EntryBar']]).strftime('%d %b %y')
        trades['entry price'] = trades['EntryPrice']
        trades['exit datetime'] = pd.to_datetime(df.index[trades['ExitBar']]).strftime('%d %b %y')
        trades['exit price'] = trades['ExitPrice']
        trades['quantity'] = trades['Size']
        trades['profit/loss value'] = trades['PnL']





        trades = trades[['name', 'entry datetime', 'entry price', 'exit datetime', 'exit price', 'quantity',
                         'profit/loss value']]


        if not os.path.isfile(output_file):
            trades.to_csv(output_file, index=False)
        else:
            trades.to_csv(output_file, mode='a', header=False, index=False)

        print(f"Trades saved to {output_file} for {file_path}")
        print(f"Sharpe Ratio: {stats['Sharpe Ratio']:.2f}")
    else:
        print(f"No trades generated for {file_path}")

def cross_validate_and_save_trades(file_paths, output_file):
    if os.path.exists(output_file):
        os.remove(output_file)

    sharpe_ratios = []
    for file_path in file_paths:
        df = load_data(file_path)
        print(f"Running backtest for: {file_path}")

        stop_loss_range = np.arange(0.01, 0.05, 0.005).tolist()
        take_profit_range = np.arange(0.02, 0.1, 0.01).tolist()

        bt = Backtest(df, DualSMA_Strategy, cash=100000, commission=0.001, exclusive_orders=True)

        stats = bt.optimize(
            stop_loss_pct=stop_loss_range,
            take_profit_pct=take_profit_range,
            maximize='Sharpe Ratio',
            constraint=lambda p: float(p.stop_loss_pct) < float(p.take_profit_pct),
            random_state=42
        )

        sharpe_ratios.append(stats['Sharpe Ratio'])
        run_backtest_and_save_trades(file_path, output_file)

    avg_sharpe_ratio = np.mean(sharpe_ratios)
    print("\nIndividual Sharpe Ratios:")
    for i, file_path in enumerate(file_paths):
        print(f"{os.path.basename(file_path)}: {sharpe_ratios[i]:.2f}")
    print(f"\nAverage Sharpe Ratio across all datasets: {avg_sharpe_ratio:.2f}")


file_paths = [
    "/content/Adani Power.csv",
    "/content/Tata Motors.csv",
    "/content/Adani Enterprises.csv",
    "/content/Adani Ports & SEZ.csv",
    "/content/Axis Bank.csv",
    "/content/DLF.csv",
    "/content/Hindalco Industries.csv",
    "/content/ICICI Bank.csv",
    "/content/Power Finance Corporation.csv",
    "/content/Punjab National Bank.csv",
    "/content/Rural Electrification Corporation.csv",
    "/content/State Bank of India.csv"
]

output_file = "optimized_trade_results.csv"
cross_validate_and_save_trades(file_paths, output_file)
print(f"\nFinal optimized trade results saved to {output_file}")
